In [2]:
model_path = '/home/LLM_para/para_glm4'
lora_path = './final_checkpoint_GLM4_SQL_SFT_cropped'
# lora_path = './final_checkpoint_newS_SFT_SQL_CN_TV'
# lora_path = './final_checkpoint_newS_SFT_SQL_CN_TV'
val_dataset_path = "./mydataset_SQL/T2Q_GLM4_SFT_val_sql.jsonl"

In [3]:
model = AutoPeftModelForCausalLM.from_pretrained(
        lora_path, trust_remote_code= True, device_map='auto')

ValueError: Can't find 'adapter_config.json' at './final_checkpoint_GLM4_SQL_SFT_cropped'

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
test_df = pd.read_json(val_dataset_path, lines=True)
print(test_df)

                                           instruction  \
0     I want you to act as a SQL terminal in front ...   
1     I want you to act as a SQL terminal in front ...   
2     I want you to act as a SQL terminal in front ...   
3     I want you to act as a SQL terminal in front ...   
4     I want you to act as a SQL terminal in front ...   
..                                                 ...   
378   I want you to act as a SQL terminal in front ...   
379   I want you to act as a SQL terminal in front ...   
380   I want you to act as a SQL terminal in front ...   
381   I want you to act as a SQL terminal in front ...   
382   I want you to act as a SQL terminal in front ...   

                                                 input  \
0    Find the name, headquarter and revenue of all ...   
1    Return the total revenue of companies with hea...   
2     Give the distinct headquarters of manufacturers.   
3    Select the names and the prices of all the pro...   
4      What i

In [6]:
def predict(messages, model, tokenizer):
    device = "cuda"
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
#     print("reponse1++++++++++++++++",response)
     
    return response

In [7]:
import re
pattern = r"-- SQL: (.*?);"
test_df_check = test_df[:10]
print(test_df_check)
test_text_list = []
for index, row in test_df_check.iterrows():
    instruction = row['instruction']
    input_value = row['input']
    messages = [
        {"role": "system", "content": f"{instruction}"},
        {"role": "user", "content": f"{input_value}"}
    ]

    response = predict(messages, model, tokenizer)
    match = re.search(pattern, response, re.DOTALL)
    if match:  
        response = match.group(1).strip() 
    else:
        sql_statement = "None"
    print("预测SQL：",response)
#     if ";" in response:
#         response = response.split(";")[0]
#     if "```SQL:" in response:
# #         print(23)
#         response = response.split("```SQL:")[1]
#     response = re.sub(r'\s+', ' ', response).strip()
# #     print(response)
    

                                         instruction  \
0   I want you to act as a SQL terminal in front ...   
1   I want you to act as a SQL terminal in front ...   
2   I want you to act as a SQL terminal in front ...   
3   I want you to act as a SQL terminal in front ...   
4   I want you to act as a SQL terminal in front ...   
5   I want you to act as a SQL terminal in front ...   
6   I want you to act as a SQL terminal in front ...   
7   I want you to act as a SQL terminal in front ...   
8   I want you to act as a SQL terminal in front ...   
9   I want you to act as a SQL terminal in front ...   

                                               input  \
0  Find the name, headquarter and revenue of all ...   
1  Return the total revenue of companies with hea...   
2   Give the distinct headquarters of manufacturers.   
3  Select the names and the prices of all the pro...   
4    What is the total revenue of each manufacturer?   
5  What is the total revenue of all companies w

In [9]:
#  read glod 
from tqdm import tqdm
# 录入db_id query用于glod  第一次使用需要
df = pd.read_csv("./mydataset_SQL/validation_dataset_formatted.csv",encoding="GBK")
results = []
for index, row in tqdm(df.iterrows(), total=len(df)):
    results.append([row['query'],row['question'],row['db_id']])
new_df = pd.DataFrame(results, columns = ['reference_query','question','db_id'])
print(new_df)

100%|██████████████████████████████████████████████████████████████████████████████| 383/383 [00:00<00:00, 11833.57it/s]

                                       reference_query  \
0    SELECT name ,  headquarter ,  revenue FROM man...   
1    SELECT SUM(revenue) FROM manufacturers WHERE h...   
2       SELECT DISTINCT headquarter FROM manufacturers   
3                   SELECT name ,  price FROM products   
4    SELECT SUM(revenue) ,  name FROM manufacturers...   
..                                                 ...   
378  SELECT name ,  built_year FROM mill WHERE type...   
379  SELECT t2.railway_id ,  t1.builder FROM railwa...   
380  SELECT objectnumber FROM railway WHERE railway...   
381  SELECT price FROM publication WHERE publisher ...   
382  SELECT publisher ,  COUNT(*) FROM publication ...   

                                              question          db_id  
0    Find the name, headquarter and revenue of all ...  manufactory_1  
1    Return the total revenue of companies with hea...  manufactory_1  
2     Give the distinct headquarters of manufacturers.  manufactory_1  
3    Select the

In [10]:
import re
index = 0
pattern = r"-- SQL: (.*?);"
predict_list = []
for index, row in test_df.iterrows():
    index += 1
    instruction = row['instruction']
    input_value = row['input']
    messages = [
        {"role": "system", "content": f"{instruction}"},
        {"role": "user", "content": f"{input_value}"}
    ] 
    response = predict(messages, model, tokenizer)
    response = predict(messages, model, tokenizer)
    match = re.search(pattern, response, re.DOTALL)
    if match:  
        response = match.group(1).strip() 
    else:
        sql_statement = "None"
    print(f"预测SQL{index}：",response)
    predict_list.append(response)
#     messages.append({"role": "assistant", "content": f"{response}"})
#     result_text = f"{messages[0]}\n\n{messages[1]}\n\n{messages[2]}"
#     print(result_text)
    
#     test_text_list.append(swanlab.Text(result_text, caption=response))

预测SQL1： SELECT name ,  headquarter ,  revenue FROM manufacturers ORDER BY revenue DESC
预测SQL2： SELECT SUM(revenue) FROM manufacturers WHERE headquarter  =  'Tokyo' OR headquarter  =  'Taiwan'
预测SQL3： SELECT DISTINCT headquarter FROM manufacturers
预测SQL4： SELECT name ,  price FROM products
预测SQL5： SELECT SUM(revenue) ,  name FROM manufacturers GROUP BY name
预测SQL6： SELECT SUM(revenue) FROM manufacturers WHERE headquarter  =  'Tokyo' OR headquarter  =  'Taiwan'
预测SQL7： SELECT SUM(revenue) FROM manufacturers WHERE headquarter  =  'Austin'
预测SQL8： SELECT name FROM products WHERE price  <=  200
预测SQL9： SELECT t1.stuid ,  COUNT(*) FROM sportsinfo as t1 JOIN plays_games as t2 ON t1.stuid  =  t2.stuid GROUP BY t1.stuid
预测SQL10： SELECT COUNT(DISTINCT stuid) FROM plays_games
预测SQL11： SELECT major ,  AVG(age) ,  MIN(age) ,  MAX(age) FROM student GROUP BY major
预测SQL12： SELECT t1.stuid ,  SUM(t1.hours_played) FROM plays_games as t1 GROUP BY t1.stuid
预测SQL13： SELECT DISTINCT gtype FROM video_games


预测SQL52： SELECT DISTINCT t3.paperid FROM paperkeyphrase as t1 JOIN paper as t3 ON t1.paperid  =  t3.paperid JOIN writes as t4 ON t4.paperid  =  t3.paperid JOIN keyphrase as t2 ON t1.keyphraseid  =  t2.keyphraseid JOIN author as t5 ON t4.authorid  =  t5.authorid WHERE t5.authorname  =  "brian curless" AND t2.keyphrasename  =  "convolution"
预测SQL53： SELECT DISTINCT t3.year FROM writes as t3 JOIN author as t2 ON t3.authorid  =  t2.authorid JOIN writes as t4 ON t4.paperid  =  t3.paperid JOIN author as t1 ON t4.authorid  =  t1.authorid WHERE t2.authorname  =  "Peter Mertens" AND t1.authorname  =  "Dina Barbian"
预测SQL54： SELECT DISTINCT t2.paperid FROM paperkeyphrase as t2 JOIN paper as t1 ON t2.paperid  =  t1.paperid JOIN keyphrase as t3 ON t2.keyphraseid  =  t3.keyphraseid WHERE t3.keyphrasename  =  "instance segmentation"
预测SQL55： SELECT DISTINCT COUNT(t1.paperid) FROM venue as t2 JOIN paper as t1 ON t2.venueid  =  t1.venueid WHERE t1.year  =  2015 AND t2.venuename  =  "nature communicati

预测SQL95： SELECT t2.fname ,  t2.lname FROM architect as t1 JOIN architect as t2 ON t1.id  =  t2.id ORDER BY t1.start_year DESC LIMIT 10
预测SQL96： SELECT milliseconds FROM tracks WHERE name = "Fast As a Shark"
预测SQL97： SELECT t2.name FROM playlist_tracks as t1 JOIN playlists as t2 ON t1.playlist_id  =  t2.id GROUP BY t1.playlist_id HAVING COUNT(t1.track_id)  >  100
预测SQL98： SELECT t1.first_name ,  t1.last_name ,  COUNT(*) FROM customers as t1 JOIN invoices as t2 ON t2.customer_id  =  t1.id GROUP BY t1.id ORDER BY COUNT(*) DESC LIMIT 10
预测SQL99： SELECT t1.first_name ,  t1.last_name FROM employees as t1 JOIN customers as t2 ON t1.id  =  t2.support_rep_id GROUP BY t1.id ORDER BY COUNT(*) DESC LIMIT 1
预测SQL100： SELECT COUNT(*) ,  t2.name FROM tracks as t1 JOIN genres as t2 ON t1.genre_id  =  t2.id GROUP BY t2.name ORDER BY COUNT(*) DESC LIMIT 5
预测SQL101： SELECT t1.title FROM albums as t1 JOIN tracks as t2 ON t1.id = t2.album_id WHERE t2.name = "Balls to the Wall"
预测SQL102： SELECT t1.name ,  t

预测SQL160： SELECT border FROM border_info WHERE state_name  =  "kentucky"
预测SQL161： SELECT density FROM state WHERE area  =  ( SELECT MIN(area) FROM state )
预测SQL162： SELECT river_name FROM river WHERE traverse != "tennessee"
预测SQL163： SELECT capital FROM state WHERE area  =  ( SELECT MAX(area) FROM state )
预测SQL164： SELECT border FROM border_info WHERE state_name IN ( SELECT border FROM border_info WHERE state_name IN ( SELECT border FROM border_info WHERE state_name IN ( SELECT border FROM border_info WHERE state_name  =  "texas" ) ) )
预测SQL165： SELECT COUNT(river_name) FROM river WHERE traverse  =  "idaho"
预测SQL166： SELECT border FROM border_info WHERE state_name  =  "kentucky"
预测SQL167： SELECT river_name FROM river WHERE traverse IN ( SELECT state_name FROM state WHERE area  =  ( SELECT MAX(area) FROM state ) )
预测SQL168： SELECT population FROM city WHERE city_name  =  "seattle" AND state_name  =  "washington"
预测SQL169： SELECT length FROM river WHERE river_name  =  "rio grande"
预测SQL

预测SQL231： SELECT t1.formats ,  COUNT(DISTINCT t2.artist_name) FROM files as t1 JOIN song as t2 ON t1.f_id  =  t2.f_id GROUP BY t1.formats
预测SQL232： SELECT MAX(home_games) ,  MIN(home_games) ,  AVG(home_games) FROM stadium
预测SQL233： SELECT DISTINCT name FROM medication ORDER BY name
预测SQL234： SELECT t1.name FROM patient as t1 JOIN appointment as t2 ON t1.ssn = t2.patient ORDER BY t2.start DESC LIMIT 1
预测SQL235： SELECT patient FROM undergoes ORDER BY dateundergoes DESC LIMIT 1
预测SQL236： SELECT COUNT(*) FROM patient EXCEPT SELECT t1.patient FROM patient as t1 JOIN prescribes as t2 ON t2.patient  =  t1.ssn JOIN medication as t3 ON t3.code  =  t2.medication WHERE t3.name  =  'Procrastin-X'
预测SQL237： SELECT t1.name ,  COUNT(*) FROM patient as t1 JOIN physician as t2 ON t1.pcp  =  t2.employeeid GROUP BY t1.pcp
预测SQL238： SELECT appointmentid FROM appointment ORDER BY start DESC LIMIT 1
预测SQL239： SELECT t2.name ,  t2.position FROM department as t1 JOIN physician as t2 ON t1.head  =  t2.employee

预测SQL286： SELECT t1.id ,  t2.installation_date FROM station as t2 JOIN trip as t1 ON t2.id  =  t1.end_station_id
预测SQL287： SELECT city FROM station ORDER BY city ,  MAX(lat) DESC
预测SQL288： SELECT start_station_id ,  start_station_name FROM trip GROUP BY start_station_name HAVING COUNT(*)  >=  200
预测SQL289： SELECT t1.id ,  t1.name FROM station as t1 JOIN status as t2 ON t1.id  =  t2.station_id WHERE t2.bikes_available  >  12 GROUP BY t1.id HAVING COUNT(*)  >  0
预测SQL290： SELECT COUNT(*) FROM station WHERE city  =  "Mountain View"
预测SQL291： SELECT name ,  lat ,  city FROM station ORDER BY lat LIMIT 1
预测SQL292： SELECT campus FROM campuses WHERE year BETWEEN 1935 AND 1939
预测SQL293： SELECT SUM(faculty) FROM faculty WHERE year  =  2002
预测SQL294： SELECT t2.campusfee FROM campuses as t1 JOIN csu_fees as t2 ON t1.id  =  t2.campus WHERE t1.campus  =  "San Francisco State University" AND t2.year  =  1996
预测SQL295： SELECT campus FROM campuses WHERE year  <  1800
预测SQL296： SELECT COUNT(*) FROM camp

预测SQL356： SELECT t2.hardware_model_name FROM chip_model as t1 JOIN phone as t2 ON t1.model_name  =  t2.chip_model JOIN screen_mode as t3 ON t3.graphics_mode  =  t2.screen_mode WHERE t3.type  =  "Text" OR t1.ram_mib  >  32
预测SQL357： SELECT t2.hardware_model_name ,  t2.company_name FROM chip_model as t1 JOIN phone as t2 ON t1.model_name  =  t2.chip_model WHERE t1.launch_year  =  2002 OR t1.ram_mib  >  32
预测SQL358： SELECT t2.name FROM repair_assignment as t1 JOIN technician as t2 ON t1.technician_id  =  t2.technician_id JOIN machine as t3 ON t1.machine_id  =  t3.machine_id ORDER BY t3.quality_rank
预测SQL359： SELECT name FROM technician ORDER BY age ASC
预测SQL360： SELECT COUNT(*) FROM technician
预测SQL361： SELECT t2.author FROM acceptance as t1 JOIN submission as t2 ON t1.submission_id  =  t2.submission_id GROUP BY t2.author HAVING COUNT(DISTINCT t1.workshop_id)  >  1
预测SQL362： SELECT college FROM submission WHERE scores  >  90 INTERSECT SELECT college FROM submission WHERE scores  <  80
预测SQ

In [11]:
new_df['generated_query'] = predict_list
new_df.to_csv("generated_SQL_GLM4_CNFilter.csv", index=False)

In [12]:
def append_string_to_file(text, file_path):
    with open(file_path, 'a') as file:
        file.write(text + '\n')

def remove_spaces(text):
    return re.sub(r'\s+', ' ', text)

In [13]:
for index, row in new_df.iterrows():
    print(f"Processing the {index}th rows")
    if pd.isna(row['generated_query']):
        print(row['generated_query'])
        sql_query = input("give me the correct SQL query")
        sql_query = remove_spaces(sql_query)
        append_string_to_file(sql_query, "Predicted_filter.txt")
        append_string_to_file(row['reference_query'] + "\t" + row['db_id'], "Gold.txt")
    elif row['generated_query'][:6] == "SELECT":
        append_string_to_file(remove_spaces(row['generated_query']), "Predicted_filter.txt")
        append_string_to_file(row['reference_query'] + "\t" + row['db_id'], "Gold.txt")
    else:
        print(row['generated_query'])
        sql_query = input("give me the correct SQL query")
        sql_query = remove_spaces(sql_query)
        append_string_to_file(sql_query, "Predicted_filter.txt")
        append_string_to_file(row['reference_query'] + "\t" + row['db_id'], "Gold.txt")

Processing the 0th rows
Processing the 1th rows
Processing the 2th rows
Processing the 3th rows
Processing the 4th rows
Processing the 5th rows
Processing the 6th rows
Processing the 7th rows
Processing the 8th rows
Processing the 9th rows
Processing the 10th rows
Processing the 11th rows
Processing the 12th rows
Processing the 13th rows
Processing the 14th rows
Processing the 15th rows
Processing the 16th rows
Processing the 17th rows
Processing the 18th rows
Processing the 19th rows
Processing the 20th rows
Processing the 21th rows
Processing the 22th rows
Processing the 23th rows
Processing the 24th rows
Processing the 25th rows
Processing the 26th rows
Processing the 27th rows
Processing the 28th rows
Processing the 29th rows
Processing the 30th rows
Processing the 31th rows
Processing the 32th rows
Processing the 33th rows
Processing the 34th rows
Processing the 35th rows
Processing the 36th rows
Processing the 37th rows
Processing the 38th rows
Processing the 39th rows
Processing